<a href="https://www.kaggle.com/code/william2020/fine-tune-qwen2-7b-on-custom-data?scriptVersionId=186973082" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Fine Tune Qwen2-7b with unsloth on custom data!

Note, this code was performed on Google Colab!

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
from google.colab import userdata

Create a Google Colab secret called **HF_TOKEN**

We'll use **userdata.get("HF_TOKEN")** in the token param below.

The below script will download the Qwen2 7b model!

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2-7b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = userdata.get('HF_TOKEN'),
)

### Make sure your're logged in to Huggingface in this notebook:

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

### Finetuning this model takes a certain format.  We'll create a formatting propmts func to assist us:

### The dataset we are loading is my specific dataset which is an assortment of question/answer pairs.

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("wzebrowski/fafsa_instruct", split = "train")

### Our Data

In [ ]:
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset

Let's add QLora

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

### Defining the Trainer

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 70,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

### Train the model (Fine Turning)

In [ ]:
trainer_stats = trainer.train()

### To save the model and tokenizer locally, run the following.

In [ ]:
# model.save_pretrained("qwen2-7b-finetuned-model") # Local saving
# tokenizer.save_pretrained("qwen2-7b-finetuned-fafsa-model")

### Run inference

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "what is fafsa?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 150, use_cache = True)
tokenizer.batch_decode(outputs)

### Push the model and tokenizer to your huggingface account.

In [ ]:
model.push_to_hub("hf/qwen2-7b-finetuned-model", token = userdata.get('HF_TOKEN')) # Online saving
tokenizer.push_to_hub("hf/qwen2-7b-finetuned-model", token = userdata.get('HF_TOKEN')) # Online saving

### You can also save the model in a quantized version and GGUF! 

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/qwen2-7b-finetuned-model", tokenizer, token = userdata.get('HF_TOKEN'))

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/qwen2-7b-finetuned-model", tokenizer, quantization_method = "f16", token = userdata.get('HF_TOKEN'))

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/qwen2-7b-finetuned-model", tokenizer, quantization_method = "q4_k_m", token = userdata.get('HF_TOKEN'))